In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
jump =pd.read_csv("/content/drive/MyDrive/Tiny_Gesture-main/data/jump_set.csv")
run =pd.read_csv("/content/drive/MyDrive/Tiny_Gesture-main/data/run_set.csv")
squat =pd.read_csv("/content/drive/MyDrive/Tiny_Gesture-main/data/squat_set.csv")
walk =pd.read_csv("/content/drive/MyDrive/Tiny_Gesture-main/data/walk_set.csv")
other =pd.read_csv("/content/drive/MyDrive/Tiny_Gesture-main/data/other_set.csv")


In [ ]:
jump.shape

In [ ]:
run.shape

In [ ]:
GESTURES=["squat_set","jump_set","walk_set","run_set","other_set"]

In [ ]:
SAMPLES_PER_GESTURE=119
NUM_GESTURES =len(GESTURES)

In [ ]:
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)
dft=[jump,run,squat,walk,other]
inputs =[]
outputs =[]

In [ ]:
#Read each csv file and push an input and output
for gesture_index in range(NUM_GESTURES):
  gesture=GESTURES[gesture_index]
  print("Processing index",{gesture_index},"for gesture",{gesture},".")
  output=ONE_HOT_ENCODED_GESTURES[gesture_index]
  print(output)
  df=dft[gesture_index]

  # Calculate the number of gesture recorings in the file
  num_recordings= int(df.shape[0]/SAMPLES_PER_GESTURE)
  print("\t There are",{num_recordings}, "recordings of the",{gesture},"gesture.")

  for i in range (num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i*SAMPLES_PER_GESTURE + j
      # Normalise the input data between 0 to 1
      # Acceleration is betwwen: -4 to +4
      # Gyroscope is between: -2000 to +2000
      tensor += [(df['aX'][index] + 4) / 8,
                (df['aY'][index] + 4) / 8,
                (df['aZ'][index] + 4) / 8,
                (df['gX'][index] + 2000) / 4000,
                (df['gY'][index] + 2000) / 4000,
                (df['gZ'][index] + 2000) / 4000
      ]

    inputs.append(tensor)
    outputs.append(output)  
# convert  the list to numpy array
inputs = np.array(inputs) 
outputs = np.array(outputs)

print("Data Set Parsing and Preparation Complete")

In [ ]:
input_shape = inputs[0].shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Dense(512,input_shape=input_shape, activation="relu"),
     tf.keras.layers.Dense(256, activation="relu"),
     tf.keras.layers.Dropout(.1),
     tf.keras.layers.Dense(128, activation="relu"),
     tf.keras.layers.Dense(64, activation="relu"),
     tf.keras.layers.Dropout(.1),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(5,activation='sigmoid')
                             
])

In [ ]:
model.compile(optimizer="adam",
              loss= 'categorical_crossentropy',
              metrics =['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, 
          epochs= 100, 
          validation_split= 0.2, 
          batch_size= 10, 
          verbose=True)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
cm = tf.math.confusion_matrix(labels=tf.argmax(y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = ['squat', 'jump','walk','run','other']
figure = plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, cmap=plt.cm.Blues, xticklabels=axis_labels, yticklabels=axis_labels)
plt.ylim(len(cm),0,0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('predicted label')
plt.show()